In [1]:
# Importamos las librerias que vamos a usar
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# largo y ancho de los ladrillos
h = 1

# Caso de tomografía acustica
N = 8    # número de observaciones
M = 16   # número de parámetros del modelo

# Construimos la matriz G
G = np.zeros((N,M), dtype=float)

# formamos la matriz kernel
for i in range(4):
    for j in range(4):
        # medidas en las lineas
            k = (i)*4 + j
            G[i][k] = h
            # medidas en las columnas
            k = (j)*4 + i
            G[i+4][k] = h
print(G)

[[1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.]
 [1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1.]]


In [4]:
# Demos valores de velocidad de onda acustica a cada ladrillo
m = np.asarray([1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4]) # km/s

#  d (s) = G (km) * m (1 / km/s)

# Calculemos la "lentitud" = 1 / velocidad
m = 1 / m

# Vector transpuesto para hacer   d = G m
# recuerden que G es 8x16 y m es 16x1 parámetros, d es 8x1 observaciones
m = m.T

# Medidas reales sin ruido
d = np.matmul(G, m)
print('Tiempos de viaje sin ruido:\n', d)
# Datos con ruido
print(d)
d_con_ruido = np.zeros(len(d), dtype=float)

# Agregaremos ruido para crear medidas más reales
for i in range(len(d)):
    d_con_ruido[i] = d[i] + np.random.normal(0,0.01)  # 0.01 segundo max desviación
print('Tiempos de viaje con ruido:\n', d_con_ruido)

Tiempos de viaje sin ruido:
 [4.         2.         1.33333333 1.         2.08333333 2.08333333
 2.08333333 2.08333333]
[4.         2.         1.33333333 1.         2.08333333 2.08333333
 2.08333333 2.08333333]
Tiempos de viaje con ruido:
 [3.99851948 1.99622644 1.32867669 1.0216084  2.09441383 2.0819155
 2.09614832 2.10251596]


In [8]:
# Calculamos G^T G
GT = G.T
GTd = np.matmul(GT, d_con_ruido)
GTG = np.matmul(GT, G)
GTGinv = np.linalg.inv(GTG)
m_est = np.matmul(GTGinv, GTd)
m_est_no_amor = m_est
print('m_real    m_est')
for i in range(M):
    print(m[i], '     ', m_est[i])


print('\n')
print('Datos observados VS sintéticos \n')
d_pred = np.matmul(G, m_est)
print('d_obs                   d_pred')
for i in range(N):
    print(d_con_ruido[i], '     ', d_pred[i])


m_real    m_est
1.0       48.0
1.0       -20.0
1.0       -46.0
1.0       -2.0
0.5       -88.0
0.5       -9.0
0.5       -2.0
0.5       56.0
0.3333333333333333       -14.0
0.3333333333333333       8.0
0.3333333333333333       4.0
0.3333333333333333       -5.0
0.25       20.0
0.25       6.0
0.25       8.0
0.25       -32.0


Datos observados VS sintéticos 

d_obs                   d_pred
3.998519478196719       -20.0
1.9962264366496645       -43.0
1.3286766902512948       -7.0
1.021608400629093       2.0
2.0944138296600503       -34.0
2.081915501030578       -15.0
2.096148315838933       -36.0
2.1025159606234887       17.0


In [9]:
print(np.linalg.det(GTG))

2.0993362836148085e-140
